# ChatGPT

#### Load documents, build the GPTVectorStoreIndex

In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index import GPTVectorStoreIndex, SimpleDirectoryReader, LLMPredictor, ServiceContext
from langchain.chat_models import ChatOpenAI
from IPython.display import Markdown, display

In [ ]:
# load documents
documents = SimpleDirectoryReader('../../data/paul_graham').load_data()

In [ ]:
# LLM Predictor (gpt-3.5-turbo) + service context
llm_predictor = LLMPredictor(llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo", streaming=True))
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, chunk_size=512)

In [ ]:
index = GPTVectorStoreIndex.from_documents(documents, service_context=service_context)

#### Query Index

By default, with the help of langchain's PromptSelector abstraction, we use 
a modified refine prompt tailored for ChatGPT-use if the ChatGPT model is used.

In [17]:
query_engine = index.as_query_engine(
    service_context=service_context,
    similarity_top_k=3,
    streaming=True,
)
response = query_engine.query(
    "What did the author do growing up?", 
)

INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 8 tokens
> [retrieve] Total embedding token usage: 8 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 0 tokens
> [get_response] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens
> [get_response] Total embedding token usage: 0 tokens


In [18]:
response.print_response_stream()

Before college, the author worked on writing short stories and programming on an IBM 1401 using an early version of Fortran. They also worked on programming with microcomputers and eventually created a new dialect of Lisp called Arc. They later realized the potential of publishing essays on the web and began writing and publishing them. The author also worked on spam filters, painting, and cooking for groups.

In [19]:
query_engine = index.as_query_engine(
    service_context=service_context,
    similarity_top_k=5,
    streaming=True,
)
response = query_engine.query(
    "What did the author do during his time at RISD?", 
)

INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 12 tokens
> [retrieve] Total embedding token usage: 12 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 0 tokens
> [get_response] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens
> [get_response] Total embedding token usage: 0 tokens


In [20]:
response.print_response_stream()

The author attended RISD and took classes in fundamental subjects like drawing, color, and design. They also learned a lot in the color class they took, but otherwise, they were basically teaching themselves to paint. The author dropped out of RISD in 1993.

**Refine Prompt**: Here is the chat refine prompt 

In [ ]:
from llama_index.prompts.chat_prompts import CHAT_REFINE_PROMPT

In [ ]:
dict(CHAT_REFINE_PROMPT.prompt)

#### Query Index (Using the standard Refine Prompt)

If we use the "standard" refine prompt (where the prompt is one text template instead of multiple messages), we find that the results over ChatGPT are worse. 

In [ ]:
from llama_index.prompts.default_prompts import DEFAULT_REFINE_PROMPT

In [ ]:
query_engine = index.as_query_engine(
    service_context=service_context,
    refine_template=DEFAULT_REFINE_PROMPT,
    similarity_top_k=5,
    streaming=True,
)
response = query_engine.query(
    "What did the author do during his time at RISD?", 
)

In [ ]:
response.print_response_stream()

### [Beta] Use ChatGPTLLMPredictor

Very simple GPT-Index-native ChatGPT wrapper. Note: this is a beta feature. If this doesn't work please
use the suggested flow above.

In [ ]:
# use ChatGPT [beta]
from llama_index.llm_predictor.chatgpt import ChatGPTLLMPredictor
from langchain.prompts.chat import SystemMessagePromptTemplate

llm_predictor = ChatGPTLLMPredictor()

In [ ]:
query_engine = index.as_query_engine(
    service_context=service_context,
    streaming=True,
)
response = query_engine.query(
    "What did the author do during his time at RISD?", 
)

In [ ]:
response.print_response_stream()